In [28]:
import os

In [29]:
os.getcwd()

'/home/bernard/GitHub/thesis/experiments/notebooks'

In [39]:
file_path = '../../data/LGL/LGL.xml'

In [37]:
import xml.etree.ElementTree as et

In [43]:
tree = et.parse(file_path)
root = tree.getroot()

In [80]:
for child in root:
    print(child.find('text').text)
    break

Alexandria woman charged in connection with Kelleyland fire. Chiquita Raquel Henry, 19, of 1935 Orchard St., Alexandria, was arrested and charged with aggravated arson and unauthorized entry of an inhabited dwelling. Both the Sheriff’s Office and Rapides Parish Fire District 2 investigated the March 7 fire at 6016 Dublin Road that led to Henry’s arrest. According to the Sheriff’s Office, the man who lives in the mobile home at that address was inside the home when Henry came in and set the bed and couch on fire with a lighter. The man only knew the woman by a nickname but investigation led detectives to Henry.


In [55]:
child.attrib

{'docid': '40450848'}

In [73]:
root[0][4].text

'2009-03-20 12:24:52.002364-05'

In [74]:
for article in root.iter('article'):
    print(article.attrib)

{'docid': '40450848'}
{'docid': '41383748'}
{'docid': '41406650'}
{'docid': '41650539'}
{'docid': '41662232'}
{'docid': '41662233'}
{'docid': '38563537'}
{'docid': '41444350'}
{'docid': '41740819'}
{'docid': '41740820'}
{'docid': '43001564'}
{'docid': '44289398'}
{'docid': '44289401'}
{'docid': '44289402'}
{'docid': '44289405'}
{'docid': '38572301'}
{'docid': '38572304'}
{'docid': '41630220'}
{'docid': '41630222'}
{'docid': '41759215'}
{'docid': '41759218'}
{'docid': '38576503'}
{'docid': '38576514'}
{'docid': '38576518'}
{'docid': '38576529'}
{'docid': '38576530'}
{'docid': '40677001'}
{'docid': '41876596'}
{'docid': '41876606'}
{'docid': '41876608'}
{'docid': '41876609'}
{'docid': '41876610'}
{'docid': '38741973'}
{'docid': '40791127'}
{'docid': '41197277'}
{'docid': '41539050'}
{'docid': '41539051'}
{'docid': '41842996'}
{'docid': '41867239'}
{'docid': '38543497'}
{'docid': '41305572'}
{'docid': '41378018'}
{'docid': '41761813'}
{'docid': '43278178'}
{'docid': '38543361'}
{'docid': 

In [ ]:
import codecs
import json
import os
import urllib
import xml.etree.ElementTree as et
import subprocess
import time
from geopy.distance import great_circle
import numpy
import matplotlib.pyplot as plt


def prepare_lgl(path_to_xml, path_to_output, directory):
    """
    Save the contents of the LGL corpus to file to create a gold corpus.
    :param directory: where to save the text files for each document in corpus
    :param path_to_xml: file containing the Local Global Lexicon annotation
    :param path_to_output: where to save the output
    :return: N/A
    """
    tree = et.parse(path_to_xml)
    root = tree.getroot()
    f = codecs.open(path_to_output, "w", "utf-8")
    c = 0
    for child in root:
        text = child.find('text').text
        gold_tops = []
        toponyms = child.findall('./toponyms/toponym')
        for top in toponyms:
            tag = top.find("gaztag")
            phrase = top.find("phrase")
            if tag is None:
                continue   # Do not include toponyms with NO coordinates, we filter these out (~17% of the dataset)
            start = top.find("start")
            end = top.find("end")
            query = {'geonameId': tag.attrib['geonameid'], 'username': "milangritta"}
            response = urllib.urlopen("http://api.geonames.org/get?" + urllib.urlencode(query))
            time.sleep(1)
            info = et.fromstring(response.read())
            area = calculate_area(info)
            print c
            name = tag.find("name")
            lat = tag.find("lat")
            lon = tag.find("lon")
            gold_tops.append(name.text + ",," + phrase.text + ",," + lat.text + ",," +
                             lon.text + ",," + start.text + ",," + end.text + ",," + str(area))
        for t in gold_tops:
            f.write(t + "||")
        f.write("\n")
        f = codecs.open("./" + directory + "/" + str(c), 'w', "utf-8")  # Files saved by numbers
        f.write(text)
        f.close()
        c += 1
    f.close()



In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger

In [12]:
tagger = SequenceTagger.load('nl-ner')

2021-01-09 14:08:04,273 https://nlp.informatik.hu-berlin.de/resources/models/nl-ner/nl-ner-bert-conll02-v0.6.pt not found in cache, downloading to /tmp/tmpcc1q8yff


100%|██████████| 502173933/502173933 [01:24<00:00, 5928448.16B/s]

2021-01-09 14:09:29,109 copying /tmp/tmpcc1q8yff to cache at /home/bernard/.flair/models/nl-ner-bert-conll02-v0.6.pt


2021-01-09 14:09:29,316 removing temp file /tmp/tmpcc1q8yff
2021-01-09 14:09:29,338 loading file /home/bernard/.flair/models/nl-ner-bert-conll02-v0.6.pt


#### Get sentence from xml file

In [14]:
file_path = '../../data/LGL/LGL.xml'

In [15]:
import xml.etree.ElementTree as et

In [16]:
tree = et.parse(file_path)
root = tree.getroot()

In [18]:
test = root[0].find('text').text

In [19]:
test

'Alexandria woman charged in connection with Kelleyland fire. Chiquita Raquel Henry, 19, of 1935 Orchard St., Alexandria, was arrested and charged with aggravated arson and unauthorized entry of an inhabited dwelling. Both the Sheriff’s Office and Rapides Parish Fire District 2 investigated the March 7 fire at 6016 Dublin Road that led to Henry’s arrest. According to the Sheriff’s Office, the man who lives in the mobile home at that address was inside the home when Henry came in and set the bed and couch on fire with a lighter. The man only knew the woman by a nickname but investigation led detectives to Henry.'

In [20]:
sentence = Sentence(test)

In [22]:
tagger.predict(sentence)

In [ ]:
for entity in sentence.get_spans('nl-ner'):
    print(entity)

In [23]:
sentence

Sentence: "Alexandria woman charged in connection with Kelleyland fire . Chiquita Raquel Henry , 19 , of 1935 Orchard St. , Alexandria , was arrested and charged with aggravated arson and unauthorized entry of an inhabited dwelling . Both the Sheriff ’s Office and Rapides Parish Fire District 2 investigated the March 7 fire at 6016 Dublin Road that led to Henry ’s arrest . According to the Sheriff ’s Office , the man who lives in the mobile home at that address was inside the home when Henry came in and set the bed and couch on fire with a lighter . The man only knew the woman by a nickname but investigation led detectives to Henry ."   [− Tokens: 118  − Token-Labels: "Alexandria <S-ORG> woman charged in connection with Kelleyland <B-MISC> fire <E-MISC> . Chiquita <B-PER> Raquel <I-PER> Henry <E-PER> , 19 , of 1935 Orchard <B-LOC> St. <E-LOC> , Alexandria <S-PER> , was arrested and charged with aggravated arson and unauthorized entry of an inhabited dwelling . Both the Sheriff <B-LOC> 

In [4]:
#Sample text to run NER
text = 'Jackson is placed in Microsoft located in Redmond'
#passing text to sentence
sentence = Sentence(text)

In [5]:
sentence

Sentence: "Jackson is placed in Microsoft located in Redmond"   [− Tokens: 8]

In [6]:
# Run NER on sentence to identify Entities
tagger.predict(sentence)
# print the entities with below command
for entity in sentence.get_spans('ner'):
    print(entity)

Span [1]: "Jackson"   [− Labels: PER (0.9999)]
Span [5]: "Microsoft"   [− Labels: ORG (0.9913)]
Span [8]: "Redmond"   [− Labels: LOC (0.9567)]


In [8]:
print(sentence.to_tagged_string())

Jackson <S-PER> is placed in Microsoft <S-ORG> located in Redmond <S-LOC>


In [11]:
#Sample text
text1 = 'Redmond is coming to New York city'
#passing text to sentence
sentence = Sentence(text1)
# Run NER on sentence to identify Entities
tagger.predict(sentence)
# print the entities with below command
for entity in sentence.get_spans('ner'):
    print(entity)

Span [1]: "Redmond"   [− Labels: PER (1.0)]
Span [5,6]: "New York"   [− Labels: LOC (0.9995)]
